In [84]:
%load_ext rpy2.ipython
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [130]:
from data_cleaning import *

In [131]:
name = "Rohit: 36"

In [132]:
extract_name(name)

NameError: name 'extract_name' is not defined

# Merge raw data files

In [86]:
# Set working directory to folder with Tangerine raw data files
# os.chdir(r'/home/rohitdaniel/Documents/Ei_CSF_FLN_Evaluation/SLO_Baseline/tangerine_files')
        
# Import the datasets that need to be merged
# mp_raw_1 = pd.read_excel("./mp_raw_data_9-25 Sep_location_fixed_v5.xlsx", index_col='_id')
# mp_raw_2 = pd.read_csv("./CSF_FLN_2022_Grade_1_Baseline-CSF_FLN_Evaluation_MP_Baseline_2022_v2.19-1666664920923.csv", index_col='_id', low_memory=False)

# up_raw_1 = pd.read_excel("./up_raw_data_19-23 Sep_location_fixed_v2.xlsx", index_col='_id')
# up_raw_2 = pd.read_csv("./CSF_FLN_2022_Grade_1_Baseline-CSF_FLN_Evaluation_UP_Baseline_2022_v2.19-1666664936961.csv", index_col='_id', low_memory=False)

In [87]:
# Flag to check if all columns in dataset1 are present in dataset 2
# all(col in mp_raw_2.columns for col in mp_raw_1.columns)

In [88]:
# missing_cols = set(mp_raw_1.columns).difference(set(mp_raw_2.columns))

In [89]:
# Rename colmns in old CSV file to facilitate concatenation
# old_cols = set(mp_raw_1.columns).difference(set(mp_raw_2.columns)) # Create list of columns that needs to be renamed

# new_cols = [re.search('^.+(?=\.\w+$)', col).group(0) for col in old_cols] # Create list of new names for the columns

# rename_cols = {} # Initialize dictionary to store old and new column names

# for old_col, new_col in zip(old_cols, new_cols):
#     rename_cols[old_col] = new_col

# mp_raw_1.rename(columns=rename_cols, inplace=True)

In [90]:
# Concatenate both raw dataframes
# mp_raw = pd.concat([mp_raw_1, mp_raw_2], join="inner")
# up_raw = pd.concat([up_raw_1, up_raw_2], join="inner")

In [91]:
# mp_fix_labels = {'school_details.UDISE_cd_label': {23310607203: 23310607101, 23310607802: 23310601105, 23310613802: 23310601104}, \
#                         'school_details.School_label': {'NMS SANKLA JAGTHAR': 'PS JAGTHAR', 'MS SANOTI':'EGS SALAIYAPURA', 'EGS TAPRA KACHHI': 'EGS KHARI TAPRA'}}

# mp_mask_idx = mp_raw.index[mp_raw.assessment_date == '2022-10-10']
# mp_raw.loc[mask_idx, :] = mp_raw.loc[mp_mask_idx, :].replace(mp_fix_labels)

In [92]:
# up_fix_labels = {'school_details.UDISE_cd_label': {9691406101: 9691405001}, \
#                  'school_details.School_label': {'P.S. SANDWA': 'P.S. DHAURUPUR'}}

# up_mask = ((up_raw.assessment_date == '2022-10-21') & ((up_raw.tabletUserName == 'Munish')  | (up_raw.tabletUserName == 'Poonamkumari')))
# up_mask_idx = up_raw.index[up_mask]

# up_raw.loc[up_mask_idx, :] = up_raw.loc[up_mask_idx, :].replace(up_fix_labels)

In [93]:
# Export concatenated dataframe to Excel file for further cleaning and analysis
# mp_raw.to_excel("mp_raw_full.xlsx")
# up_raw.to_excel("up_raw_full.xlsx")

# Data Cleaning Functions

In [94]:
def toDatetime(dataframe):
    time_cols = [col for col in dataframe.columns if re.search(r'^(?!Submission).*(?!_)(?i)time$', col)]

    for col in time_cols:
        dataframe.loc[:, col] = dataframe.apply(lambda row: datetime.fromtimestamp(float(row[col])/1000) if not ((pd.isna(row[col])) | (row[col] == 'UNDEFINED') | (row[col] == 'SKIPPED')) else row[col], axis=1)
    return None

In [95]:
def update_or_scores(dataframe, other_responses):
    return None

In [96]:
def outlier_treatment(dataframe, score_col):
    iqr = np.percentile(dataframe[score_col], 75) - np.percentile(dataframe[score_col], 25)
    outliers = dataframe[dataframe[score_col] >= 1.5*iqr]
    return (outliers)

# Import merged raw data file

In [97]:
# Change working directory
os.chdir(r'/home/rohitdaniel/Documents/CSF_FLN_Evaluation/SLO_Baseline/rawData')

# Import concatenated raw dataframe for cleaning
mp_raw = pd.read_excel("./mp_raw_full.xlsx", index_col='_id')
mp_raw.name = 'mp_raw'

up_raw = pd.read_excel("./up_raw_full.xlsx", index_col='_id', )
up_raw.name = 'up_raw'

## A. Literacy Sub-tasks Data Cleaning

In [98]:
mp_raw_lit, mp_raw_num = clean_data(mp_raw)
mp_raw_lit.name = 'MP literacy'
mp_raw_num.name = 'MP numeracy'

In [99]:
up_raw_lit, up_raw_num = clean_data(up_raw)
up_raw_lit.name = 'UP literacy'
up_raw_num.name = 'UP numeracy'

In [101]:
for dataframe in [mp_raw_lit, up_raw_lit]:
    duration_cols = [col for col in dataframe.columns if re.search(r'.+time_remaining$', col)]
    clean_scores(dataframe, duration_cols)

### Literacy 1: Listening Comprehension

In [102]:
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit1_raw = [col for col in dataframe.columns if re.search(r'literacy1_q\d$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 1 scores\033[0m')
    clean_scores(dataframe, lit1_raw)
    
    
    # Calculate total score and percentage correct on listening comprehension sub-task
    dataframe.loc[:, 'literacy1_total'] = dataframe.apply(lambda x: total_score([x[col] for col in lit1_raw]), axis=1)
    dataframe.loc[:, 'literacy1_%_correct'] = dataframe.apply(lambda x: 100*x['literacy1_total']/len(lit1_raw), axis=1)
    
    # Extract other responses to listening comprehension questions
    lit1_or = [col for col in dataframe.columns if re.search(r'^literacy1_\S*or$', col)]
    file_name = dataframe.name + '_lit1_or.xlsx'
    with pd.ExcelWriter(file_name) as writer: 
        for col in lit1_or:
            dataframe[col].value_counts().reset_index().rename(columns = {'index':"Response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 2: Oral Vocabulary
</h3>

In [103]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit2_raw = [col for col in dataframe.columns if re.search(r'literacy2_q\d+$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 2 scores\033[0m')
    clean_scores(dataframe, lit2_raw)
        
    dataframe.loc[:, 'literacy2_total'] = dataframe.apply(lambda x: total_score([x[col] for col in lit2_raw]), axis=1)
    dataframe.loc[:, 'literacy2_%_correct'] = dataframe.apply(lambda x: 100*x['literacy2_total']/len(lit2_raw), axis=1)
    
    # Extract other responses to oral vocabulary questions
    lit2_or = [col for col in dataframe.columns if re.search(r'literacy2\S*or$', col)]
    file_name = dataframe.name + '_lit2_or.xlsx'
    with pd.ExcelWriter(file_name) as writer: 
        for col in lit2_or:
            dataframe[col].value_counts().reset_index().rename(columns = {'index':"Response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 3: Initial Sound Identification
</h3>

In [104]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit3_raw = [col for col in dataframe.columns if re.search(r'literacy3_q\d+$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 3 scores\033[0m')
    clean_scores(dataframe, lit3_raw)
    
    dataframe.loc[:, 'literacy3_total'] = dataframe.apply(lambda x: total_score([x[col] for col in lit3_raw]), axis=1)
    dataframe.loc[:, 'literacy3_%_correct'] = dataframe.apply(lambda x: 100*x['literacy3_total']/len(lit3_raw), axis=1)

<h3>
    Literacy 4: Letter Recognition (Untimed)
</h3>

In [105]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit4_ut_raw = [col for col in dataframe.columns if re.search(r'literacy4_ut_grid_\d*$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 4 (untimed) scores\033[0m')
    clean_scores(dataframe, lit4_ut_raw)
    dataframe.loc[:, lit4_ut_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit4_ut_raw]), axis=1)).to_list(), index=dataframe.index, columns=lit4_ut_raw)
    
    # Calculate total score on letter naming (untimed) sub-task
    dataframe.loc[:, 'literacy4_ut_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit4_ut_raw]), axis=1)
    dataframe.loc[:, 'literacy4_ut_%_correct'] = dataframe.apply(lambda x: 100*x['literacy4_ut_total']/len(lit4_ut_raw), axis=1)

### Literacy 4: Letter Recognition (Timed)

In [106]:
def fluency_scores(dataframe, cols):
    fluency = 0
    score_cols = [col for col in cols if re.search(r'.+\d$', col)]
    total = total_score(dataframe[score_cols])
    duration_col = [col for col in cols if re.search(r'.+time_remaining$', col)]
    duration = (60 - int(dataframe[duration_col]))
    if duration != 0:
        fluency = ((60*total)/duration)
    return fluency

In [115]:
# Check data for 'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit4_tt_raw = [col for col in dataframe.columns if re.search(r'literacy4_tt.+', col)]
    lit4_tt_scores = [col for col in dataframe.columns if re.search(r'literacy4_tt_grid_\d+$', col)]

    # print(f'\033[1mCleaning literacy {dataframe.name} subtask 4 (timed) scores\033[0m')
    clean_scores(dataframe, lit4_tt_scores)
    dataframe.loc[:, lit4_tt_scores] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit4_tt_scores]), axis=1)).to_list(), index=dataframe.index, columns=lit4_tt_scores)
    # Calculate automaticity on letter naming (timed) sub-task
    dataframe.loc[:, 'literacy4_tt_fluency'] = dataframe.apply(lambda x: fluency_score(x, lit4_tt_raw), axis=1)

NameError: name 're' is not defined

<h3>
    Literacy 5: Familiar Words Reading (Untimed)
</h3>

In [45]:
# Check data for 'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit5_ut_raw = [col for col in dataframe.columns if re.search(r'literacy5_ut_grid_\d*$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 5 (untimed) scores\033[0m')
    clean_scores(dataframe, lit5_ut_raw)
    dataframe.loc[:, lit5_ut_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit5_ut_raw]), axis=1)).to_list(), index=dataframe.index, columns=lit5_ut_raw)
    
    dataframe.loc[:, 'literacy5_ut_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit5_ut_raw]), axis=1)
    dataframe.loc[:, 'literacy5_ut_%_correct'] = dataframe.apply(lambda x: 100*x['literacy5_ut_total']/len(lit5_ut_raw), axis=1)

<h3>
    Literacy 5: Familiar Words Reading (Timed)
</h3>

In [46]:
# Check data for 'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit5_tt_raw = [col for col in dataframe.columns if re.search(r'literacy5_tt.+', col)]
    lit5_tt_scores = [col for col in dataframe.columns if re.search(r'literacy5_tt_grid_\d+$', col)]

    # print(f'\033[1mCleaning literacy {dataframe.name} subtask 5 (timed) scores\033[0m')
    clean_scores(dataframe, lit5_tt_scores)
    dataframe.loc[:, lit5_tt_scores] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit5_tt_scores]), axis=1)).to_list(), index=dataframe.index, columns=lit5_tt_scores)
    
    # Calculate automaticity on letter naming (timed) sub-task
#     dataframe.loc[:, 'literacy5_tt_fluency'] = dataframe.apply(lambda x: fluency_score(x, lit5_tt_raw), axis=1)

### Literacy 6: Non-word Reading

In [50]:
# Check data for 'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit6_tt_raw = [col for col in dataframe.columns if re.search(r'literacy6_tt.+', col)]
    lit6_tt_scores = [col for col in dataframe.columns if re.search(r'literacy6_tt_grid_\d+$', col)]
    #     print(f'\033[1mCleaning literacy {dataframe.name} subtask 6 scores\033[0m')
    clean_scores(dataframe, lit6_tt_scores)
    dataframe.loc[:, lit6_tt_scores] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit6_tt_scores]), axis=1)).to_list(), index=dataframe.index, columns=lit6_tt_scores)
    
    # Calculate automaticity on letter naming (timed) sub-task
#     dataframe.loc[:, 'literacy6_tt_fluency'] = dataframe.apply(lambda x: fluency_score(x, lit6_tt_raw), axis=1)

### Literacy 7: Oral Reading Fluency (Timed)

In [52]:
# Check data for 'UNDEFINED' or 'SKIPPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit7_raw = [col for col in dataframe.columns if re.search(r'literacy7_tt.+', col)]
    lit7_scores = [col for col in dataframe.columns if re.search(r'literacy7_tt_grid_\d+$', col)]
    
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 7 scores\033[0m')
    clean_scores(dataframe, lit7_scores)
    dataframe.loc[:, lit7_scores] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit7_scores]), axis=1)).to_list(), index=dataframe.index, columns=lit7_scores)
    
     # Calculate automaticity on letter naming (timed) sub-task
#     dataframe.loc[:, 'literacy7_tt_fluency'] = dataframe.apply(lambda x: fluency_score(x, lit7_raw), axis=1)

### Literacy 8: Reading Comprehension (Untimed)

In [55]:
# Check data for 'UNDEFINED' or 'SKIPPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit8_raw_reading = [col for col in dataframe.columns if re.search(r'literacy8_ut_grid_\d*$', col)]
    lit8_raw_comprehension = [col for col in dataframe.columns if re.search(r'literacy8_ut_q\d*$', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 8 scores\033[0m')
    for scores in [lit8_raw_reading, lit8_raw_comprehension]:
        clean_scores(dataframe, scores)
    dataframe.loc[:, lit8_raw_reading] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit8_raw_reading]), axis=1)).to_list(), index=dataframe.index, columns=lit8_raw_reading)
    
    dataframe.loc[:, 'literacy8_reading_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit8_raw_reading]), axis=1)
    dataframe.loc[:, 'literacy8_reading_%_correct'] = dataframe.apply(lambda x: 100*x['literacy8_reading_total']/len(lit8_raw_reading), axis=1)
    dataframe.loc[:, 'literacy8_comprehension_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit8_raw_comprehension]), axis=1)
    dataframe.loc[:, 'literacy8_comprehension_%_correct'] = dataframe.apply(lambda x: 100*x['literacy8_comprehension_total']/len(lit8_raw_comprehension), axis=1)

### Literacy 9a: Dictation (Letters)

In [57]:
# Check data for 'UNDEFINED' or 'SKIPPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit9a_raw = [col for col in dataframe.columns if re.search(r'literacy9a_ut_grid_\d*', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 9a scores\033[0m')
    clean_scores(dataframe, lit9a_raw)
    dataframe.loc[:, lit9a_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit9a_raw]), axis=1)).to_list(), index=dataframe.index, columns=lit9a_raw)
    dataframe.loc[:, 'literacy9a_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit9a_raw]), axis=1)
    dataframe.loc[:, 'literacy9a_%_correct'] = dataframe.apply(lambda x: 100*x['literacy9a_total']/len(lit9a_raw), axis=1)
    # dataframe.loc[:, 'literacy9a_total'] = dataframe.apply(lambda x: validated_scores(x, mp_validated), axis=1)

In [58]:
mp_dictation_letters_validation = mp_raw_lit[mp_raw_lit.literacy9a_total >= 5][['assessment_date', 'school_details.State_label', 'school_details.District_label', \
                                              'school_details.Block_label', 'school_details.School_label', \
                                              'school_details.UDISE_cd_label', 'SI_std_name', 'GI_std_name', 'student_age', \
                                              'student_gender', 'literacy9a_total']]

In [59]:
up_dictation_letters_validation = up_raw_lit[up_raw_lit.literacy9a_total >= 5][['assessment_date', 'school_details.State_label', 'school_details.District_label', \
                                              'school_details.Block_label', 'school_details.School_label', \
                                              'school_details.UDISE_cd_label', 'SI_std_name', 'GI_std_name', 'student_age', \
                                              'student_gender', 'literacy9a_total']]

### Literacy 9b: Dictation (Words)

In [60]:
# Check data for 'UNDEFINED' or 'SKIPPPED' values
for dataframe in [mp_raw_lit, up_raw_lit]:
    lit9b_raw = [col for col in dataframe.columns if re.search(r'literacy9b_ut_grid_\d*', col)]
#     print(f'\033[1mCleaning literacy {dataframe.name} subtask 9b scores\033[0m')
    clean_scores(dataframe, lit9b_raw)
    dataframe.loc[:, lit9b_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_scores([x[col] for col in lit9b_raw]), axis=1)).to_list(), index=dataframe.index, columns=lit9b_raw)
    
    dataframe.loc[:, 'literacy9b_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit9b_raw]), axis=1)
    dataframe.loc[:, 'literacy9b_%_correct'] = dataframe.apply(lambda x: 100*x['literacy9b_total']/len(lit9b_raw), axis=1)


In [61]:
mp_dictation_words_validation = mp_raw_lit[mp_raw_lit.literacy9b_total >= 5][['assessment_date', 'school_details.State_label', 'school_details.District_label', \
                                              'school_details.Block_label', 'school_details.School_label', \
                                              'school_details.UDISE_cd_label', 'SI_std_name', 'GI_std_name', 'student_age', \
                                              'student_gender', 'literacy9b_total']]

In [62]:
up_dictation_words_validation = up_raw_lit[up_raw_lit.literacy9b_total >= 5][['assessment_date', 'school_details.State_label', 'school_details.District_label', \
                                              'school_details.Block_label', 'school_details.School_label', \
                                              'school_details.UDISE_cd_label', 'SI_std_name', 'GI_std_name', 'student_age', \
                                              'student_gender', 'literacy9b_total']]

In [63]:
mp_dictation_validation = pd.merge(left=mp_dictation_letters_validation, right=mp_dictation_words_validation, how='outer')
mp_dictation_validation.to_excel('mp_dictation_validation.xlsx')

In [64]:
up_dictation_validation = pd.merge(left=up_dictation_letters_validation, right=up_dictation_words_validation, how='outer')
up_dictation_validation.to_excel('up_dictation_validation.xlsx')

/home/rohitdaniel/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [66]:
for dataframe in [mp_raw_lit, up_raw_lit]:
    file_name = dataframe.name + '_cleaned.xlsx'
    dataframe.loc[:, cols].to_excel(file_name)

KeyError: "['literacy4_tt_fluency', 'literacy5_tt_fluency', 'literacy6_tt_fluency', 'literacy7_tt_fluency'] not in index"

In [487]:
mp_raw_lit['student_gender'].nunique()

4

<h2>
    B. Numeracy Sub-tasks Data Cleaning
</h2>

<h3>
    Numeracy 1: Counting
</h3>

In [39]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_num, up_raw_num]:
    num1_raw = [col for col in dataframe.columns if re.search(r'numeracy1_tt_grid_\d*$', col)]
    print(f'\033[1mCleaning numeracy {dataframe.name} subtask 1 (timed) scores\033[0m')
    clean_scores(num1_raw, dataframe)
    dataframe.loc[:, num1_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_counting_score([x[col] for col in num1_raw]), axis=1)).to_list(), index=dataframe.index, columns=num1_raw)
    
#     dataframe.loc[:, 'literacy5_ut_total'] = dataframe.apply(lambda x: total_score([x[score] for score in lit5_ut_raw]), axis=1)

Cleaning numeracy MP numeracy subtask 1 (timed) scores
No. of NaN values in numeracy1_tt_grid_1 = 0
No. of UNDEFINED values in numeracy1_tt_grid_1 = 0
No. of SKIPPED values in numeracy1_tt_grid_1 = 0
Unique values in numeracy1_tt_grid_1 = ['1' '0']

No. of NaN values in numeracy1_tt_grid_2 = 0
No. of UNDEFINED values in numeracy1_tt_grid_2 = 0
No. of SKIPPED values in numeracy1_tt_grid_2 = 0
Unique values in numeracy1_tt_grid_2 = ['1' '999' '0']

No. of NaN values in numeracy1_tt_grid_3 = 0
No. of UNDEFINED values in numeracy1_tt_grid_3 = 0
No. of SKIPPED values in numeracy1_tt_grid_3 = 0
Unique values in numeracy1_tt_grid_3 = ['1' '999' '0']

No. of NaN values in numeracy1_tt_grid_4 = 0
No. of UNDEFINED values in numeracy1_tt_grid_4 = 0
No. of SKIPPED values in numeracy1_tt_grid_4 = 0
Unique values in numeracy1_tt_grid_4 = ['1' '0' '999']

No. of NaN values in numeracy1_tt_grid_5 = 0
No. of UNDEFINED values in numeracy1_tt_grid_5 = 0
No. of SKIPPED values in numeracy1_tt_grid_5 = 0
Un

<h3>
    Numeracy 2: Number Recognition (Untimed)
</h3>

In [40]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_num, up_raw_num]:
    num2_ut_raw = [col for col in dataframe.columns if re.search(r'numeracy2_ut_grid_\d*$', col)]
    print(f'\033[1mCleaning numeracy {dataframe.name} subtask 2 (untimed) scores\033[0m')
    clean_scores(num2_ut_raw, dataframe)
    dataframe.loc[:, num2_ut_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_score([x[col] for col in num2_ut_raw]), axis=1)).to_list(), index=dataframe.index, columns=num2_ut_raw)
    
    dataframe.loc[:, 'numeracy2_ut_total'] = dataframe.apply(lambda x: total_score([x[score] for score in num2_ut_raw]), axis=1)

Cleaning numeracy MP numeracy subtask 2 (untimed) scores
No. of NaN values in numeracy2_ut_grid_1 = 0
No. of UNDEFINED values in numeracy2_ut_grid_1 = 0
No. of SKIPPED values in numeracy2_ut_grid_1 = 0
Unique values in numeracy2_ut_grid_1 = ['0' '1']

No. of NaN values in numeracy2_ut_grid_2 = 0
No. of UNDEFINED values in numeracy2_ut_grid_2 = 0
No. of SKIPPED values in numeracy2_ut_grid_2 = 0
Unique values in numeracy2_ut_grid_2 = ['0' '1']

No. of NaN values in numeracy2_ut_grid_3 = 0
No. of UNDEFINED values in numeracy2_ut_grid_3 = 0
No. of SKIPPED values in numeracy2_ut_grid_3 = 0
Unique values in numeracy2_ut_grid_3 = ['0' '1']

No. of NaN values in numeracy2_ut_grid_4 = 0
No. of UNDEFINED values in numeracy2_ut_grid_4 = 0
No. of SKIPPED values in numeracy2_ut_grid_4 = 0
Unique values in numeracy2_ut_grid_4 = ['0' '1' '999']

No. of NaN values in numeracy2_ut_grid_5 = 0
No. of UNDEFINED values in numeracy2_ut_grid_5 = 0
No. of SKIPPED values in numeracy2_ut_grid_5 = 0
Unique value

<h3>
    Numeracy 2: Number Recognition (Timed)
</h3>

In [64]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for dataframe in [mp_raw_num, up_raw_num]:
    num2_ut_raw = [col for col in dataframe.columns if re.search(r'numeracy2_ut_grid_\d*$', col)]
    print(f'\033[1mCleaning numeracy {dataframe.name} subtask 2 (untimed) scores\033[0m')
    clean_scores(num2_ut_raw, dataframe)
    dataframe.loc[:, num2_ut_raw] = pd.DataFrame((dataframe.apply(lambda x: fix_score([x[col] for col in num2_ut_raw]), axis=1)).to_list(), index=dataframe.index, columns=num2_ut_raw)
    
    dataframe.loc[:, 'numeracy2_ut_total'] = dataframe.apply(lambda x: total_score([x[score] for score in num2_ut_raw]), axis=1)
numeracy2_tt_raw = [col for col in numeracy2_tt if re.search(r'numeracy2_tt_grid_', col)]


Unique values in numeracy2_tt_grid_1 = ['0' '1' 1 0]
No. of NaN values in numeracy2_tt_grid_1 = 0
No. of UNDEFINED values in numeracy2_tt_grid_1 = 0
No. of SKIPPED values in numeracy2_tt_grid_1 = 0

Unique values in numeracy2_tt_grid_2 = ['0' '1' '.']
No. of NaN values in numeracy2_tt_grid_2 = 0
No. of UNDEFINED values in numeracy2_tt_grid_2 = 0
No. of SKIPPED values in numeracy2_tt_grid_2 = 0

Unique values in numeracy2_tt_grid_3 = ['0' '1' '.']
No. of NaN values in numeracy2_tt_grid_3 = 0
No. of UNDEFINED values in numeracy2_tt_grid_3 = 0
No. of SKIPPED values in numeracy2_tt_grid_3 = 0

Unique values in numeracy2_tt_grid_4 = ['0' '1' '.']
No. of NaN values in numeracy2_tt_grid_4 = 0
No. of UNDEFINED values in numeracy2_tt_grid_4 = 0
No. of SKIPPED values in numeracy2_tt_grid_4 = 0

Unique values in numeracy2_tt_grid_5 = ['.' '1' '0']
No. of NaN values in numeracy2_tt_grid_5 = 0
No. of UNDEFINED values in numeracy2_tt_grid_5 = 0
No. of SKIPPED values in numeracy2_tt_grid_5 = 0

Uniqu

In [65]:
mp_numeracy.loc[:, numeracy2_tt_raw] = pd.DataFrame((mp_numeracy.apply(lambda x: fix_score([x[col] for col in numeracy2_tt_raw]), axis=1)).to_list(), index=mp_numeracy.index, columns=numeracy2_tt_raw)

<h3>
    Numeracy 3: Number Comparison
</h3>

In [66]:
# Check data for'UNDEFINED' or 'SKIPPED' values
for col in numeracy3:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy3_ut_q1 = ['1' '0' '88' 1 0]
No. of NaN values in numeracy3_ut_q1 = 0
No. of UNDEFINED values in numeracy3_ut_q1 = 0
No. of SKIPPED values in numeracy3_ut_q1 = 0

Unique values in numeracy3_ut_q2 = ['0' '1' '88' 1 0]
No. of NaN values in numeracy3_ut_q2 = 0
No. of UNDEFINED values in numeracy3_ut_q2 = 0
No. of SKIPPED values in numeracy3_ut_q2 = 0

Unique values in numeracy3_ut_q3 = ['1' '88' '0' 0 1]
No. of NaN values in numeracy3_ut_q3 = 0
No. of UNDEFINED values in numeracy3_ut_q3 = 0
No. of SKIPPED values in numeracy3_ut_q3 = 0

Unique values in numeracy3_ut_q4 = ['1' '0' '88' 0 1]
No. of NaN values in numeracy3_ut_q4 = 0
No. of UNDEFINED values in numeracy3_ut_q4 = 0
No. of SKIPPED values in numeracy3_ut_q4 = 0

Unique values in numeracy3_ut_q5 = ['1' '0' nan '88' 0 1]
No. of NaN values in numeracy3_ut_q5 = 339
No. of UNDEFINED values in numeracy3_ut_q5 = 0
No. of SKIPPED values in numeracy3_ut_q5 = 0

Unique values in numeracy3_ut_q6 = ['0' '1' nan '88' 0

<h3>
    Numeracy 4: Counting in Bundles
</h3>

In [67]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy4_raw = [col for col in numeracy4 if re.search(r'numeracy4.+\d$', col)]
for col in numeracy4_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy4_ut_q1 = ['0' '1' '88' 0 88 1]
No. of NaN values in numeracy4_ut_q1 = 0
No. of UNDEFINED values in numeracy4_ut_q1 = 0
No. of SKIPPED values in numeracy4_ut_q1 = 0

Unique values in numeracy4_ut_q2 = ['88' '0' '1' 0 88]
No. of NaN values in numeracy4_ut_q2 = 0
No. of UNDEFINED values in numeracy4_ut_q2 = 0
No. of SKIPPED values in numeracy4_ut_q2 = 0

Unique values in numeracy4_ut_q3 = ['88' '0' '1' 0 1 88]
No. of NaN values in numeracy4_ut_q3 = 0
No. of UNDEFINED values in numeracy4_ut_q3 = 0
No. of SKIPPED values in numeracy4_ut_q3 = 0

Unique values in numeracy4_ut_q4 = ['88' '0' '1' 0 88]
No. of NaN values in numeracy4_ut_q4 = 0
No. of UNDEFINED values in numeracy4_ut_q4 = 0
No. of SKIPPED values in numeracy4_ut_q4 = 0



<h3>
    Numeracy 5: Missing Numbers
</h3>

In [68]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy5_raw = [col for col in numeracy5 if re.search(r'numeracy5.+\d$', col)]
for col in numeracy5_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy5_ut_q1 = ['88' '1' '0' 1 88 0]
No. of NaN values in numeracy5_ut_q1 = 0
No. of UNDEFINED values in numeracy5_ut_q1 = 0
No. of SKIPPED values in numeracy5_ut_q1 = 0

Unique values in numeracy5_ut_q2 = ['88' '1' '0' 88 0]
No. of NaN values in numeracy5_ut_q2 = 0
No. of UNDEFINED values in numeracy5_ut_q2 = 0
No. of SKIPPED values in numeracy5_ut_q2 = 0

Unique values in numeracy5_ut_q3 = ['88' '1' '0' 1 88 0]
No. of NaN values in numeracy5_ut_q3 = 0
No. of UNDEFINED values in numeracy5_ut_q3 = 0
No. of SKIPPED values in numeracy5_ut_q3 = 0

Unique values in numeracy5_ut_q4 = ['88' '0' '1' 0 88 1]
No. of NaN values in numeracy5_ut_q4 = 0
No. of UNDEFINED values in numeracy5_ut_q4 = 0
No. of SKIPPED values in numeracy5_ut_q4 = 0

Unique values in numeracy5_ut_q5 = [nan '0' '1' '88' 88 0]
No. of NaN values in numeracy5_ut_q5 = 1098
No. of UNDEFINED values in numeracy5_ut_q5 = 0
No. of SKIPPED values in numeracy5_ut_q5 = 0

Unique values in numeracy5_ut_q6 = [nan '0

<h3>
    Numeracy 6: Addition
</h3>

In [69]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy6_raw = [col for col in numeracy6 if re.search(r'numeracy6.+\d$', col)]
for col in numeracy6_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy6_ut_q1 = ['88' '1' '0' 0 88 1]
No. of NaN values in numeracy6_ut_q1 = 0
No. of UNDEFINED values in numeracy6_ut_q1 = 0
No. of SKIPPED values in numeracy6_ut_q1 = 0

Unique values in numeracy6_ut_q2 = ['88' '0' '1' 0 88]
No. of NaN values in numeracy6_ut_q2 = 0
No. of UNDEFINED values in numeracy6_ut_q2 = 0
No. of SKIPPED values in numeracy6_ut_q2 = 0

Unique values in numeracy6_ut_q3 = ['88' '1' '0' 1 88 0]
No. of NaN values in numeracy6_ut_q3 = 0
No. of UNDEFINED values in numeracy6_ut_q3 = 0
No. of SKIPPED values in numeracy6_ut_q3 = 0

Unique values in numeracy6_ut_q4 = ['88' '1' '0' 0 88 1]
No. of NaN values in numeracy6_ut_q4 = 0
No. of UNDEFINED values in numeracy6_ut_q4 = 0
No. of SKIPPED values in numeracy6_ut_q4 = 0

Unique values in numeracy6_ut_q5 = [nan '1' '88' '0' 0 88]
No. of NaN values in numeracy6_ut_q5 = 972
No. of UNDEFINED values in numeracy6_ut_q5 = 0
No. of SKIPPED values in numeracy6_ut_q5 = 0

Unique values in numeracy6_ut_q6 = [nan '1'

<h3>
    Numeracy 7: Subtraction
</h3>

In [70]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy7_raw = [col for col in numeracy7 if re.search(r'numeracy7.+\d$', col)]
for col in numeracy7_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy7_ut_q1 = ['88' '0' '1' 1 88 0]
No. of NaN values in numeracy7_ut_q1 = 0
No. of UNDEFINED values in numeracy7_ut_q1 = 0
No. of SKIPPED values in numeracy7_ut_q1 = 0

Unique values in numeracy7_ut_q2 = ['88' '0' '1' 0 88 1]
No. of NaN values in numeracy7_ut_q2 = 0
No. of UNDEFINED values in numeracy7_ut_q2 = 0
No. of SKIPPED values in numeracy7_ut_q2 = 0

Unique values in numeracy7_ut_q3 = ['88' '0' '1' 0 88]
No. of NaN values in numeracy7_ut_q3 = 0
No. of UNDEFINED values in numeracy7_ut_q3 = 0
No. of SKIPPED values in numeracy7_ut_q3 = 0

Unique values in numeracy7_ut_q4 = ['88' '0' '1' 1 88 0]
No. of NaN values in numeracy7_ut_q4 = 0
No. of UNDEFINED values in numeracy7_ut_q4 = 0
No. of SKIPPED values in numeracy7_ut_q4 = 0

Unique values in numeracy7_ut_q5 = [nan '0' '1' '88' 0]
No. of NaN values in numeracy7_ut_q5 = 1051
No. of UNDEFINED values in numeracy7_ut_q5 = 0
No. of SKIPPED values in numeracy7_ut_q5 = 0

Unique values in numeracy7_ut_q6 = [nan '0' '

<h3>
    Numeracy 8: Word Problems
</h3>

In [71]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy8_raw = [col for col in numeracy8 if re.search(r'numeracy8.+\d$', col)]
for col in numeracy8_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy.loc[:, col].fillna('999', inplace=True)
    mp_numeracy[col].replace('UNDEFINED', '999', inplace=True)
    mp_numeracy[col].replace('SKIPPED', '999', inplace=True)
    mp_numeracy[col].replace('.', '999', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy8_ut_q1 = ['1' '88' '0' 1 88 0]
No. of NaN values in numeracy8_ut_q1 = 0
No. of UNDEFINED values in numeracy8_ut_q1 = 0
No. of SKIPPED values in numeracy8_ut_q1 = 0

Unique values in numeracy8_ut_q2 = ['88' '1' '0' 0 1 88]
No. of NaN values in numeracy8_ut_q2 = 0
No. of UNDEFINED values in numeracy8_ut_q2 = 0
No. of SKIPPED values in numeracy8_ut_q2 = 0

Unique values in numeracy8_ut_q3 = ['1' '88' '0' 0 88 1]
No. of NaN values in numeracy8_ut_q3 = 0
No. of UNDEFINED values in numeracy8_ut_q3 = 0
No. of SKIPPED values in numeracy8_ut_q3 = 0

Unique values in numeracy8_ut_q4 = ['0' '1' '88' 0 88 1]
No. of NaN values in numeracy8_ut_q4 = 0
No. of UNDEFINED values in numeracy8_ut_q4 = 0
No. of SKIPPED values in numeracy8_ut_q4 = 0

Unique values in numeracy8_ut_q5 = ['0' nan '88' '1' 1 0 88]
No. of NaN values in numeracy8_ut_q5 = 844
No. of UNDEFINED values in numeracy8_ut_q5 = 0
No. of SKIPPED values in numeracy8_ut_q5 = 0

Unique values in numeracy8_ut_q6 = ['0'

<h3>
    Numeracy 9a: Shape Recognition (Circle)
</h3>

In [72]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy9a_raw = [col for col in numeracy9a if re.search(r'numeracy9a_ut_grid_', col)]
for col in numeracy9a_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy[col].replace('UNDEFINED', '0', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy9a_ut_grid_1 = ['1' '0' 'UNDEFINED' 1 0]
No. of NaN values in numeracy9a_ut_grid_1 = 0
No. of UNDEFINED values in numeracy9a_ut_grid_1 = 2
No. of SKIPPED values in numeracy9a_ut_grid_1 = 0

Unique values in numeracy9a_ut_grid_2 = ['1' '0' 'UNDEFINED' 1 0]
No. of NaN values in numeracy9a_ut_grid_2 = 0
No. of UNDEFINED values in numeracy9a_ut_grid_2 = 2
No. of SKIPPED values in numeracy9a_ut_grid_2 = 0

Unique values in numeracy9a_ut_grid_3 = ['1' '0' 'UNDEFINED' 1 0]
No. of NaN values in numeracy9a_ut_grid_3 = 0
No. of UNDEFINED values in numeracy9a_ut_grid_3 = 2
No. of SKIPPED values in numeracy9a_ut_grid_3 = 0

Unique values in numeracy9a_ut_grid_4 = ['0' '1' 'UNDEFINED' 1 0]
No. of NaN values in numeracy9a_ut_grid_4 = 0
No. of UNDEFINED values in numeracy9a_ut_grid_4 = 2
No. of SKIPPED values in numeracy9a_ut_grid_4 = 0

Unique values in numeracy9a_ut_grid_5 = ['1' '0' 'UNDEFINED' 1 0]
No. of NaN values in numeracy9a_ut_grid_5 = 0
No. of UNDEFINED values in n

<h3>
    Numeracy 9b: Shape Recognition (Rectangle)
</h3>

In [73]:
# Check data for'UNDEFINED' or 'SKIPPED' values
numeracy9b_raw = [col for col in numeracy9b if re.search(r'numeracy9b_ut_grid_', col)]
for col in numeracy9b_raw:
    print(f"Unique values in {col} = {mp_numeracy[col].unique()}")
    print(f"No. of NaN values in {col} = {mp_numeracy.loc[:, col].isna().sum()}")
    print(f"No. of UNDEFINED values in {col} = {mp_numeracy[mp_numeracy[col] == 'UNDEFINED'].shape[0]}")    
    print(f"No. of SKIPPED values in {col} = {mp_numeracy[mp_numeracy[col] == 'SKIPPED'].shape[0]}\n")
    mp_numeracy[col].replace('UNDEFINED', '0', inplace=True)
    mp_numeracy.loc[:, col] = mp_numeracy.loc[:, col].astype('str')

Unique values in numeracy9b_ut_grid_1 = ['1' '0' 1 0]
No. of NaN values in numeracy9b_ut_grid_1 = 0
No. of UNDEFINED values in numeracy9b_ut_grid_1 = 0
No. of SKIPPED values in numeracy9b_ut_grid_1 = 0

Unique values in numeracy9b_ut_grid_2 = ['1' '0' 0 1]
No. of NaN values in numeracy9b_ut_grid_2 = 0
No. of UNDEFINED values in numeracy9b_ut_grid_2 = 0
No. of SKIPPED values in numeracy9b_ut_grid_2 = 0

Unique values in numeracy9b_ut_grid_3 = ['0' '1' 1 0]
No. of NaN values in numeracy9b_ut_grid_3 = 0
No. of UNDEFINED values in numeracy9b_ut_grid_3 = 0
No. of SKIPPED values in numeracy9b_ut_grid_3 = 0

Unique values in numeracy9b_ut_grid_4 = ['1' '0' 1 0]
No. of NaN values in numeracy9b_ut_grid_4 = 0
No. of UNDEFINED values in numeracy9b_ut_grid_4 = 0
No. of SKIPPED values in numeracy9b_ut_grid_4 = 0

Unique values in numeracy9b_ut_grid_5 = ['1' '0' 0 1]
No. of NaN values in numeracy9b_ut_grid_5 = 0
No. of UNDEFINED values in numeracy9b_ut_grid_5 = 0
No. of SKIPPED values in numeracy9b_

In [74]:
mp_numeracy.to_excel("mp_raw_numeracy_full.xlsx")